# Filtering rows 2: Using `filter` and the Expression API

By the end of this lecture you will be able to:
- select rows with the `filter` method
- add a row number column
- paritioning a `DataFrame`

The `filter` method is our first example of the *Expression API*.

_**Learning to use the *Expression API* is the most important step to writing high performance queries in Polars**_


In [ ]:
import polars as pl

In [ ]:
csvFile = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csvFile)
df.head(3)

## Selecting rows with `filter`

We first use an *expression* in the `filter` method before we examine the syntax in more detail.

In this example we choose all the first class passengers.

In [ ]:
(
    df
    .filter(
        pl.col('Pclass') == 1
    )
    .head(2)
)

## Syntax of `filter`
Inside the `filter` method we pass our first _**expression**_ and apply a Boolean condition to it:

`pl.col('Pclass') == 1`

This expression has two parts:
- `pl.col('Pclass')` expression selects the `Pclass` column from `df`
- `== 1` applies a Boolean condition to this expression

In this example we choose all rows with the number of parents & children (`Parch`) is greater than 1

In [ ]:
(
    df
    .filter(
        pl.col('Parch') > 1
    )
    .head(2)
)

As well as the mathemtical operators such as `==`,`>`,`<` there are also text operators that some people find more readable

In [ ]:
(
    df
    .filter(
        pl.col('Parch').gt(1)
    )
    .head(3)
)

You can see the operators here: https://pola-rs.github.io/polars/py-polars/html/reference/expressions/operators.html

## Use cases of `[]` and `filter`

In general we use `filter` to apply a condition. In Polars you cannot filter using `[]` with a Boolean mask so the following will raise an `Exception`:
```python
df[df["Parch"] > 1]
```

The `[]` approach is primarily for selecting data by row number. This might be when inspecting data in interactive mode or retrieving a value at the end of a query

### Conditions based on row numbers with `filter`

We can add an explicit row number column using `with_row_count` on a `DataFrame`

In [ ]:
df = pl.read_csv(csvFile)
df = df.with_row_count(name='row_nr')
df.head(3)

We can then use `filter` to apply a condition based on row number

In [ ]:
(
    df
    .filter(
        pl.col('row_nr') < 4
    )
)

### Filtering on a Boolean column
We can filter for `True` values on a Boolean column by passing the column as an expression to `filter`

In [ ]:
(
    df
    .with_columns(
        less_than_30 = pl.col("Age") < 30
    )
    .filter(
        pl.col("less_than_30")
    )
    .head(2)
)

We can negate a filter with `~`

In [ ]:
(
    df
    .with_columns(
        less_than_30 = pl.col("Age") < 30
    )
    .filter(
        ~pl.col("less_than_30")
    )
    .head(2)
)

## Partitioning a `DataFrame`
In some cases we want to filter on one or more columns repeatedly. 

For example, when I create an interactive visualisation app I have a dropdown box to select different subsets of the `DataFrame`. In this case it can be useful to make a dictionary mapping from the values in the filter column(s) to the subset of the `DataFrame`.

We can do this partition into sub-`DataFrames` with the `partition_by` method 

In [ ]:
df_pclass_dict = (
    df
    .partition_by("Pclass",as_dict=True)
)

The output is a python `dict` mapping from the unique values in `Pclass` to the sub-`DataFrame` for each class. This requires copying the rows in `df` to new sub-`DataFrames`.

Note that if we don't pass the `as_dict=True` argument we get a python `list` of sub-`DataFrames`.

We can get the rows with first class passengers from this `dict`

In [ ]:
df_pclass_dict[1].head(2)

## Filter in lazy mode
Create a `LazyFrame` by scanning the CSV and adding a `filter` operation

In [ ]:
(
    pl.scan_csv(csvFile)
    .filter(pl.col("Age") > 30)
)

When we print the optimized plan we see the `filter` operation after the CSV is scanned

In [ ]:
print(
    pl.scan_csv(csvFile)
    .filter(pl.col("Age") > 30)
    .explain()
)

We see that Polars reads the CSV and then applies the filter.

If we set `streaming=True` in `explain` we see that the `filter` operation is inside the 
```
--- PIPELINE
--- END PIPELINE
```
part of the query plan - this means that Polars can do this filter operation in streaming mode if we evaluate the lazy query with `.collect(streaming=True)`

In [ ]:
print(
    pl.scan_csv(csvFile)
    .filter(pl.col("Age") > 30)
    .explain(streaming=True)
)

# Exercises
In the exercises you will develop your understanding of
- using the `filter` method
- adding a row number column
- partitioning a `DataFrame`

### Exercise 1 
Select all rows where `Age` is greater than 30

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

Select all rows where `Embarked` is equal to "C" - use the text operator rather than the mathematical operator for this

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

### Exercise 2 

In this exercise we filter on row numbers.

First add a row number column

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

Continue by selecting the first 5 rows using `filter` on the row number column

### Exercise 3
Partition the `DataFrame` by the `Survived` and `Pclass` columns as a `dict` (you may want to check the API docs for help: https://pola-rs.github.io/polars/py-polars/html/reference/dataframe/api/polars.DataFrame.partition_by.html#polars.DataFrame.partition_by)

In [ ]:
survived_pclass_dict = (
    pl.read_csv(csvFile)
    <blank>
)

Return the sub-`DataFrame` with the passengers who did not survive from the third class

## Solutions

### Solution to Exercise 1
Select all rows with `Age` greater than 30

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(pl.col('Age') > 30)
    .head(3)
)

### Solution to Exercise 2
Add a row number column

In [ ]:
(
    pl.read_csv(csvFile)
    .with_row_count("row_nr")
)

Continue by selecting the first 5 rows using `filter` on the row number column

In [ ]:
(
    pl.read_csv(csvFile)
    .with_row_count("row_nr")
    .filter(pl.col("row_nr")<5)
)

### Solution to Exercise 3
Partition the `DataFrame` by the `Survived` and `Pclass` columns as a `dict`

In [ ]:
survived_pclass_dict = (
    pl.read_csv(csvFile)
    .partition_by("Survived","Pclass",as_dict=True)
)

In [ ]:
survived_pclass_dict.keys()

Return the sub-`DataFrame` with the passengers who did not survive from the third class

In [ ]:
(
    survived_pclass_dict[(0,3)]
    .head(2)
)